In [1]:
from sympy import symbols, acos, diff, sin, cos,Derivative,pi

In [2]:
%matplotlib  inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
from plot_animate import frequency, plot_mol_mode

In [3]:
plt.rcParams['figure.dpi'] = 150
plt.rcParams['savefig.dpi'] = 150

three body structure (one dimension)
<img src="./figure/co2.png" alt="Drawing" style="width: 200px;"/>
https://en.wikipedia.org/wiki/Carbon_dioxide

In [4]:
# System CO2#

'''
O1:1
C:2
O2:3
'''

# Stiffness #
KCO1 = 1600
KCO2 = 1600

stiffness_list = np.array([KCO1,KCO2])
#spring_length_list = np.array([LOH1,L0H2])


# Mass (index: 1,2....n) #
MO1 = 16
MC = 12
MO2 = 16


mass_list = np.array([MO1,MC,MO2])

# Position (equilibrium length)#
PO1 = [0,0]
PC = [1.163,0]
PO2 = [2.326,0]


position_list = np.array([PO1,PC,PO2])

# Bond list #

bond_list = np.array([[0,1],[1,2]])


In [5]:
def bond_stiffness_1D(k,m1,m2):
    return np.array([[k/(m1**0.5*m1**0.5),-k/(m1**0.5*m2**0.5)],[-k/(m1**0.5*m2**0.5),k/(m2**0.5*m2**0.5)]])

In [6]:
def hessian_1D(m_list,p_list,b_list,k_list):
    hm = np.zeros((m_list.shape[0],m_list.shape[0]))
    
    num_cout = 0
    for i,j in bond_list:
        k_n = bond_stiffness_1D(k_list[num_cout],m_list[i],m_list[j])
        
        hm[i,i] += k_n[0,0]
        hm[i,j] += k_n[0,1]
        
        hm[j,i] += k_n[1,0]
        hm[j,j] += k_n[1,1]
        
    return hm

In [7]:
system_kmatrix_co2 = hessian_1D(mass_list,position_list,bond_list,stiffness_list)

In [8]:
# hessian matrix (stiffness matrix)
print(system_kmatrix_co2)

[[ 100.         -115.47005384    0.        ]
 [-115.47005384  266.66666667 -115.47005384]
 [   0.         -115.47005384  100.        ]]


In [9]:
# solve eignvalue and eignvector #
system_co2_val, system_co2_vec = np.linalg.eig(system_kmatrix_co2)
print(system_co2_val)
print(system_co2_vec)

[ 3.66666667e+02  1.00000000e+02 -9.69188486e-15]
[[-3.69274473e-01 -7.07106781e-01  6.03022689e-01]
 [ 8.52802865e-01  2.20031596e-16  5.22232968e-01]
 [-3.69274473e-01  7.07106781e-01  6.03022689e-01]]


Frequency
<img src="./figure/freq.png" alt="Drawing" style="width: 200px;"/>
https://www.colby.edu/chemistry/PChem/notes/NormalModesText.pdf

In [21]:
# Acorrding different eignvalue (frequency) to select different eignvector (modesharp) #
modeshape = 1
plot_mol_mode(position_list,frequency(system_co2_val[modeshape]),system_co2_vec[:,modeshape])

three body structure (two dimensions)
<img src="./figure/question.png" alt="Drawing" style="width: 200px;"/>
https://www.khanacademy.org/science/ap-biology/chemistry-of-life/structure-of-water-and-hydrogen-bonding/a/hs-water-and-life-review

In [11]:
# System H2O# 
# strength and bending #

'''
H1:0
O:1
H2:2

'''

# Stiffness #
KOH1 = 450
KOH2 = 450
KHOH = 55

#LOH1 = 0.5
#LOH2 = 0.5

bond_stiffness_list = np.array([KOH1,KOH2])
angle_stiffness_list = np.array([KHOH])
#spring_length_list = np.array([LOH1,L0H2])


# Mass #
MH1 = 1
MH2 = 1
MO = 16

mass_list = np.array([MH1,MO,MH2])

# Position #
PH1 = [0,0]
PO = [0.757,0.58]
PH2 = [1.5139,0]

position_list = np.array([PH1,PO,PH2])

# Bond list #

bond_list = np.array([[0,1],[1,2]])
angle_list = np.array([[0,1,2]])

In [12]:
def bond_stiffness_2D(k,m1,m2):
    return np.array([[k/(m1**0.5*m1**0.5),0,-k/(m1**0.5*m2**0.5),0],[0,k/(m1**0.5*m1**0.5),0,-k/(m1**0.5*m2**0.5)],[-k/(m1**0.5*m2**0.5),0,k/(m2**0.5*m2**0.5),0],[0,-k/(m1**0.5*m2**0.5),0,k/(m2**0.5*m2**0.5)]])

<font size=4>
$$ \frac{\partial K_{angle}}{\partial S} = 2K_{angle}\left(\theta-\theta_0  \right)\frac{\partial \theta}{\partial S}  $$

    
$$ \frac{\partial^2 K_{angle}}{\partial S \partial T} = 2K_{angle}\left(\left(\frac{\partial \theta}{\partial S}\frac{\partial \theta}{\partial T} \right) + \left(\theta-\theta_0  \right)\frac{\partial^2 \theta}{\partial S \partial T} \right)$$

In [13]:
# 前面像沒有加到 #

def angle_stiffness_2d(k,t0,m1,m2,m3,p1,p2,p3):
    
    
    
    xi,yi,xj,yj,xk,yk = symbols('xi yi xj yj xk yk', real=True)
    cos_theta = (((xj-xi)**2+(yj-yi)**2)+((xk-xj)**2+(yk-yj)**2)-((xk-xi)**2+(yk-yi)**2))/(2*((xj-xi)**2+(yj-yi)**2)**0.5+((xk-xj)**2+(yk-yj)**2)**0.5)
    
    potential_form = k*(acos(cos_theta)-t0)**2
    
    theta_xi_xi = (potential_form.diff(xi).diff(xi)).subs(xi,p1[0]).subs(yi,p1[1]).subs(xj,p2[0]).subs(yj,p2[1]).subs(xk,p3[0]).subs(yk,p3[1])
    theta_yi_yi = (potential_form.diff(yi).diff(yi)).subs(xi,p1[0]).subs(yi,p1[1]).subs(xj,p2[0]).subs(yj,p2[1]).subs(xk,p3[0]).subs(yk,p3[1])
    theta_xi_yi = (potential_form.diff(xi).diff(yi)).subs(xi,p1[0]).subs(yi,p1[1]).subs(xj,p2[0]).subs(yj,p2[1]).subs(xk,p3[0]).subs(yk,p3[1])  
    theta_xj_xj = (potential_form.diff(xj).diff(xj)).subs(xi,p1[0]).subs(yi,p1[1]).subs(xj,p2[0]).subs(yj,p2[1]).subs(xk,p3[0]).subs(yk,p3[1])
    theta_yj_yj = (potential_form.diff(yj).diff(yj)).subs(xi,p1[0]).subs(yi,p1[1]).subs(xj,p2[0]).subs(yj,p2[1]).subs(xk,p3[0]).subs(yk,p3[1])
    theta_xj_yj = (potential_form.diff(xj).diff(yj)).subs(xi,p1[0]).subs(yi,p1[1]).subs(xj,p2[0]).subs(yj,p2[1]).subs(xk,p3[0]).subs(yk,p3[1])   
    theta_xk_xk = (potential_form.diff(xk).diff(xk)).subs(xi,p1[0]).subs(yi,p1[1]).subs(xj,p2[0]).subs(yj,p2[1]).subs(xk,p3[0]).subs(yk,p3[1])
    theta_yk_yk = (potential_form.diff(yk).diff(yk)).subs(xi,p1[0]).subs(yi,p1[1]).subs(xj,p2[0]).subs(yj,p2[1]).subs(xk,p3[0]).subs(yk,p3[1])
    theta_xk_yk = (potential_form.diff(xk).diff(yk)).subs(xi,p1[0]).subs(yi,p1[1]).subs(xj,p2[0]).subs(yj,p2[1]).subs(xk,p3[0]).subs(yk,p3[1])    
    theta_xi_xj = (potential_form.diff(xi).diff(xj)).subs(xi,p1[0]).subs(yi,p1[1]).subs(xj,p2[0]).subs(yj,p2[1]).subs(xk,p3[0]).subs(yk,p3[1])
    theta_xi_xk = (potential_form.diff(xi).diff(xk)).subs(xi,p1[0]).subs(yi,p1[1]).subs(xj,p2[0]).subs(yj,p2[1]).subs(xk,p3[0]).subs(yk,p3[1])
    theta_yi_yj = (potential_form.diff(yi).diff(yj)).subs(xi,p1[0]).subs(yi,p1[1]).subs(xj,p2[0]).subs(yj,p2[1]).subs(xk,p3[0]).subs(yk,p3[1])
    theta_yi_yk = (potential_form.diff(yi).diff(yk)).subs(xi,p1[0]).subs(yi,p1[1]).subs(xj,p2[0]).subs(yj,p2[1]).subs(xk,p3[0]).subs(yk,p3[1])
    theta_xi_yj = (potential_form.diff(xi).diff(yj)).subs(xi,p1[0]).subs(yi,p1[1]).subs(xj,p2[0]).subs(yj,p2[1]).subs(xk,p3[0]).subs(yk,p3[1])
    theta_yi_xj = (potential_form.diff(yi).diff(xj)).subs(xi,p1[0]).subs(yi,p1[1]).subs(xj,p2[0]).subs(yj,p2[1]).subs(xk,p3[0]).subs(yk,p3[1])
    theta_yi_xk = (potential_form.diff(yi).diff(xk)).subs(xi,p1[0]).subs(yi,p1[1]).subs(xj,p2[0]).subs(yj,p2[1]).subs(xk,p3[0]).subs(yk,p3[1])
    theta_xi_yk = (potential_form.diff(xi).diff(yk)).subs(xi,p1[0]).subs(yi,p1[1]).subs(xj,p2[0]).subs(yj,p2[1]).subs(xk,p3[0]).subs(yk,p3[1])   
    theta_xj_xk = (potential_form.diff(xj).diff(xk)).subs(xi,p1[0]).subs(yi,p1[1]).subs(xj,p2[0]).subs(yj,p2[1]).subs(xk,p3[0]).subs(yk,p3[1])
    theta_yj_yk = (potential_form.diff(yj).diff(yk)).subs(xi,p1[0]).subs(yi,p1[1]).subs(xj,p2[0]).subs(yj,p2[1]).subs(xk,p3[0]).subs(yk,p3[1])    
    theta_xj_yk = (potential_form.diff(xj).diff(yk)).subs(xi,p1[0]).subs(yi,p1[1]).subs(xj,p2[0]).subs(yj,p2[1]).subs(xk,p3[0]).subs(yk,p3[1])
    theta_yj_xk = (potential_form.diff(yj).diff(xk)).subs(xi,p1[0]).subs(yi,p1[1]).subs(xj,p2[0]).subs(yj,p2[1]).subs(xk,p3[0]).subs(yk,p3[1])
    
    '''
    angle_local = np.array([[theta_xi_xi/(m1**0.5*m1**0.5),theta_xi_yi/(m1**0.5*m1**0.5),theta_xi_xj/(m1**0.5*m2**0.5),theta_xi_yj/(m1**0.5*m2**0.5),theta_xi_xk/(m1**0.5*m2**0.5),theta_xi_yk/(m1**0.5*m3**0.5)],
                            [theta_xi_yi/(m1**0.5*m1**0.5),theta_yi_yi/(m1**0.5*m1**0.5),theta_yi_xj/(m1**0.5*m2**0.5),theta_yi_yj/(m1**0.5*m2**0.5),theta_yi_xk/(m1**0.5*m3**0.5),theta_yi_yk/(m1**0.5*m3**0.5)],
                            [theta_xi_xj/(m2**0.5*m1**0.5),theta_yi_xj/(m2**0.5*m1**0.5),theta_xj_xj/(m2**0.5*m2**0.5),theta_xj_yj/(m2**0.5*m2**0.5),theta_xj_xk/(m2**0.5*m3**0.5),theta_xj_yk/(m2**0.5*m3**0.5)],
                            [theta_xi_yj/(m2**0.5*m1**0.5),theta_yi_yj/(m2**0.5*m1**0.5),theta_xj_yj/(m2**0.5*m2**0.5),theta_yj_yj/(m2**0.5*m2**0.5),theta_yj_xk/(m2**0.5*m3**0.5),theta_yj_yk/(m2**0.5*m3**0.5)],
                            [theta_xi_xk/(m1**0.5*m3**0.5),theta_yi_xk/(m1**0.5*m3**0.5),theta_xj_xk/(m2**0.5*m3**0.5),theta_yj_xk/(m2**0.5*m3**0.5),theta_xk_xk/(m3**0.5*m3**0.5),theta_xk_yk/(m3**0.5*m3**0.5)],
                            [theta_xi_yk/(m1**0.5*m3**0.5),theta_yi_yk/(m1**0.5*m3**0.5),theta_xj_yk/(m2**0.5*m3**0.5),theta_yj_yk/(m2**0.5*m3**0.5),theta_xk_yk/(m3**0.5*m3**0.5),theta_yk_yk/(m3**0.5*m3**0.5)]])
    
    '''
    angle_local = np.array([[theta_xi_xi/(m1**0.5*m1**0.5),0,theta_xi_xj/(m1**0.5*m2**0.5),0,theta_xi_xk/(m1**0.5*m2**0.5),0],
                            [0,theta_yi_yi/(m1**0.5*m1**0.5),0,theta_yi_yj/(m1**0.5*m2**0.5),0,theta_yi_yk/(m1**0.5*m3**0.5)],
                            [theta_xi_xj/(m2**0.5*m1**0.5),0,theta_xj_xj/(m2**0.5*m2**0.5),0,theta_xj_xk/(m2**0.5*m3**0.5),0],
                            [0,theta_yi_yj/(m2**0.5*m1**0.5),0,theta_yj_yj/(m2**0.5*m2**0.5),0,theta_yj_yk/(m2**0.5*m3**0.5)],
                            [theta_xi_xk/(m1**0.5*m3**0.5),0,theta_xj_xk/(m2**0.5*m3**0.5),0,theta_xk_xk/(m3**0.5*m3**0.5),0],
                            [0,theta_yi_yk/(m1**0.5*m3**0.5),0,theta_yj_yk/(m2**0.5*m3**0.5),0,theta_yk_yk/(m3**0.5*m3**0.5)]])
    
    return angle_local.astype('float')
    
    
    

In [14]:
def hessian_2D(m_list,p_list,b_list,a_list,kb_list,ka_list):
    hm = np.zeros((m_list.shape[0]*2,m_list.shape[0]*2))
    
    num_cout = 0
    for i,j in b_list*2:

        k_n = bond_stiffness_2D(kb_list[num_cout],m_list[int(i/2)],m_list[int(j/2)])
        #print(k_n)
        hm[i,i:i+2] += k_n[0,0:2]
        hm[i,j:j+2] += k_n[0,2:4]
        
        hm[i+1,i:i+2] += k_n[1,0:2]
        hm[i+1,j:j+2] += k_n[1,2:4]
        
        hm[j,i:i+2] += k_n[2,0:2]
        hm[j,j:j+2] += k_n[2,2:4]
        
        hm[j+1,i:i+2] += k_n[3,0:2]
        hm[j+1,j:j+2] += k_n[3,2:4]
        
        num_cout += 1
    
    
    num_cout = 0
    for i,j,k in a_list*2:
        ka_n = angle_stiffness_2d(ka_list[num_cout],np.deg2rad(104.52),m_list[int(i/2)],m_list[int(j/2)],m_list[int(k/2)],p_list[int(i/2)],p_list[int(j/2)],p_list[int(k/2)])
        
        hm[i,i:i+2] += ka_n[0,0:2]
        hm[i,j:j+2] += ka_n[0,2:4]
        hm[i,k:k+2] += ka_n[0,4:6]
        
        hm[i+1,i:i+2] += ka_n[1,0:2]
        hm[i+1,j:j+2] += ka_n[1,2:4]
        hm[i+1,k:k+2] += ka_n[1,4:6]
        
        hm[j,i:i+2] += ka_n[2,0:2]
        hm[j,j:j+2] += ka_n[2,2:4]
        hm[j,k:k+2] += ka_n[2,4:6]
        
        hm[j+1,i:i+2] += ka_n[3,0:2]
        hm[j+1,j:j+2] += ka_n[3,2:4]
        hm[j+1,k:k+2] += ka_n[3,4:6]
        
        hm[k,i:i+2] += ka_n[4,0:2]
        hm[k,j:j+2] += ka_n[4,2:4]
        hm[k,k:k+2] += ka_n[4,4:6]
        
        hm[k+1,i:i+2] += ka_n[5,0:2]
        hm[k+1,j:j+2] += ka_n[5,2:4]
        hm[k+1,k:k+2] += ka_n[5,4:6]
        
        num_cout += 1
     
     
    return hm

In [15]:
system_kmatrix_h2o = hessian_2D(mass_list,position_list,bond_list,angle_list,bond_stiffness_list,angle_stiffness_list)

In [16]:
system_h2o_val, system_h2o_vec = np.linalg.eig(system_kmatrix_h2o)

In [17]:
system_h2o_val

array([  0.85782943, 492.27544847, 519.34659369,   0.        ,
       443.74065622, 560.78452652])

In [18]:
system_h2o_vec

array([[ 2.28870187e-01, -8.66385106e-01,  7.97725166e-01,
         2.57293838e-16, -5.54002996e-16,  3.10843609e-17],
       [ 5.40836966e-18,  3.86785923e-17,  4.65006196e-17,
         2.35702260e-01,  7.01500732e-01,  6.72563132e-01],
       [ 9.44365286e-01,  9.80118455e-02, -3.22964990e-01,
         1.08703779e-15, -1.15321470e-16,  8.03604416e-17],
       [-0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         9.42809042e-01,  2.79063411e-03, -3.33321652e-01],
       [ 2.36204664e-01,  4.89663686e-01,  5.09242746e-01,
         2.85929932e-16,  6.37951128e-16, -7.50235883e-16],
       [-0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         2.35702260e-01, -7.12663269e-01,  6.60723474e-01]])

In [19]:
# Acorrding different eignvalue (frequency) to select different eignvector (modesharp) #
modeshape = 0
plot_mol_mode(position_list,frequency(system_h2o_val[modeshape]),system_h2o_vec[:,modeshape],boundary=[-2, 3,-1, 1],dim=2)

In [20]:
frequency(system_h2o_val[5])

3085.081502509547